In [596]:
import json
# First, read the file
trips_file = open('trips.json')
# Then, convert contents to list of dictionaries 
trips = json.load(trips_file)

In [597]:
len(trips)

1000

In [598]:
trips[0].keys()

dict_keys(['dropoff_datetime', 'dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'imp_surcharge', 'mta_tax', 'passenger_count', 'payment_type', 'pickup_datetime', 'pickup_latitude', 'pickup_longitude', 'rate_code', 'tip_amount', 'tolls_amount', 'total_amount', 'trip_distance', 'vendor_id'])

In [599]:
def parse_trips(trips):                                #https://stackoverflow.com/a/52880427/10322075
    tgt={'pickup_latitude',
        'pickup_longitude',
        'trip_distance'}    
    return [{k:v for k,v in d.items() if k in tgt} for d in trips]

def parse_trips(trips): 
    return list(map(lambda trip: {'trip_distance': trip['trip_distance'], 'pickup_latitude': trip['pickup_latitude'], 'pickup_longitude': trip['pickup_longitude']},trips))

In [600]:
parsed_trips = parse_trips(trips)
parsed_trips and parsed_trips[0]

# {'pickup_latitude': '40.64499',
#  'pickup_longitude': '-73.78115',
#  'trip_distance': '18.38'}

{'trip_distance': '18.379999999999999',
 'pickup_latitude': '40.64499',
 'pickup_longitude': '-73.781149999999997'}

In [601]:
def float_values(trips): 
    return list(map(lambda trip: {k:float(v) for k, v in trip.items()}, parsed_trips))

In [602]:
cleaned_trips = float_values(parsed_trips)

In [603]:
cleaned_trips[0]

# {'pickup_latitude': 40.64499,
#  'pickup_longitude': -73.78115,
#  'trip_distance': 18.38}

{'trip_distance': 18.38,
 'pickup_latitude': 40.64499,
 'pickup_longitude': -73.78115}

In [604]:
import folium
manhattan_map = folium.Map(location=[40.7589, -73.9851], zoom_start=11)

In [605]:
manhattan_map

In [606]:
marker = folium.CircleMarker(location = [40.7589, -73.9851], radius=10)
marker.add_to(manhattan_map)

In [607]:
manhattan_map

In [608]:
first_trip = {'pickup_latitude': 40.64499, 'pickup_longitude': -73.78115,  'trip_distance': 18.38}
first_trip

{'pickup_latitude': 40.64499,
 'pickup_longitude': -73.78115,
 'trip_distance': 18.38}

In [609]:
def location(trip): 
    return [trip['pickup_latitude'], trip['pickup_longitude']]

In [610]:
first_location = location(first_trip) # [40.64499, -73.78115]
first_location # [40.64499, -73.78115]

[40.64499, -73.78115]

In [611]:
def to_marker(location):
    return folium.CircleMarker(location, radius=6)

In [612]:
import json
times_square_marker = to_marker(location(cleaned_trips[0]))

In [613]:
times_square_marker

In [614]:
times_square_marker and times_square_marker.location # [40.7589, -73.9851]

[40.64499, -73.78115]

In [615]:
times_square_marker and json.loads(times_square_marker.options)['radius'] # 6

6

In [616]:
def markers_from_trips(trips):
    return list(map(lambda trip : to_marker(location(trip)), trips))

In [617]:
trip_markers = markers_from_trips(cleaned_trips)

In [618]:
cleaned_trips[0:4]

[{'trip_distance': 18.38,
  'pickup_latitude': 40.64499,
  'pickup_longitude': -73.78115},
 {'trip_distance': 1.3,
  'pickup_latitude': 40.766931,
  'pickup_longitude': -73.982098},
 {'trip_distance': 4.5,
  'pickup_latitude': 40.77773,
  'pickup_longitude': -73.951902},
 {'trip_distance': 2.4,
  'pickup_latitude': 40.795678,
  'pickup_longitude': -73.971049}]

In [619]:
trip_markers and len(trip_markers) # 1000

1000

In [620]:
list(map(lambda marker: marker.location, trip_markers[0:4]))
# [[40.64499, -73.78115],
#  [40.766931, -73.982098],
#  [40.77773, -73.951902],
#  [40.795678, -73.971049]]

[[40.64499, -73.78115],
 [40.766931, -73.982098],
 [40.77773, -73.951902],
 [40.795678, -73.971049]]

In [621]:
def map_from(location, zoom_amount):
    return folium.Map(location=location, zoom_start=zoom_amount)

In [622]:
times_square_map = map_from([40.7589, -73.9851], 15)

In [623]:
times_square_map and times_square_map.location # [40.7589, -73.9851]

[40.7589, -73.9851]

In [624]:
times_square_map and times_square_map.zoom_start # 15

15

In [625]:
times_square_marker and times_square_marker.add_to(times_square_map)
times_square_map

In [626]:
manhattan_map = map_from([40.7589, -73.9851], 13)

In [627]:
def add_markers(markers, map_obj):
    for marker in markers: marker.add_to(map_obj)
    return map_obj

In [628]:
map_with_markers = add_markers(trip_markers, manhattan_map)

In [629]:
map_with_markers

In [630]:
import math

def distance_location(selected_trip, neighbor_trip):
    ydistance = abs(selected_trip['pickup_latitude'] - (neighbor_trip['pickup_latitude']))
    xdistance = abs(selected_trip['pickup_longitude'] - (neighbor_trip['pickup_longitude']))
    csquared = pow(ydistance, 2) + pow(xdistance, 2)
    c = math.sqrt(csquared)
    return c

In [631]:
first_trip = {'pickup_latitude': 40.64499, 'pickup_longitude': -73.78115, 'trip_distance': 18.38}
second_trip = {'pickup_latitude': 40.766931, 'pickup_longitude': -73.982098, 'trip_distance': 1.3}
distance_first_and_second = distance_location(first_trip, second_trip)

distance_first_and_second and round(distance_first_and_second, 3) # 0.235

0.235

In [632]:
def distance_between_neighbors(selected_trip, neighbor_trip):
    neighbor_with_distance = neighbor_trip.copy()
    neighbor_with_distance['distance_from_selected'] = distance_first_and_second #update dictionary w/ new key 'distance'
    return neighbor_with_distance
    
    #return map(lambda trip : trip.update({'distance_from_selected': distance_first_and_second}), trips)
    #alt one-line code: neighbor_trip.update({'distance_from_selected': distance_first_and_second})

In [633]:
distance_between_neighbors(first_trip, second_trip)

# {'distance_from_selected': 0.23505256047318146,
#  'pickup_latitude': 40.766931,
#  'pickup_longitude': -73.982098,
#  'trip_distance': 1.3}

{'pickup_latitude': 40.766931,
 'pickup_longitude': -73.982098,
 'trip_distance': 1.3,
 'distance_from_selected': 0.23505256047318146}

In [634]:
def distance_all(selected_individual, neighbors):                        # provided a list of neighbors, returns each of those neighbors with their respective distance_from_selected numbers.
    remaining_neighbors = filter(lambda neighbor: neighbor != selected_individual, neighbors)
    return list(map(lambda neighbor : distance_between_neighbors(selected_individual, neighbor), remaining_neighbors))

In [635]:
cleaned_trips and distance_all(first_trip, cleaned_trips[0:4])

[{'trip_distance': 1.3,
  'pickup_latitude': 40.766931,
  'pickup_longitude': -73.982098,
  'distance_from_selected': 0.23505256047318146},
 {'trip_distance': 4.5,
  'pickup_latitude': 40.77773,
  'pickup_longitude': -73.951902,
  'distance_from_selected': 0.23505256047318146},
 {'trip_distance': 2.4,
  'pickup_latitude': 40.795678,
  'pickup_longitude': -73.971049,
  'distance_from_selected': 0.23505256047318146}]

In [636]:
def nearest_neighbors(selected_trip, trips, number = 3):              ######## INCORRECT ########
    neighbor_distances = distance_all(selected_trip, trips)
    sorted_neighbors = sorted(neighbor_distances, key=lambda neighbor: neighbor['distance_from_selected'])
    return sorted_neighbors[:number]

In [637]:
new_trip = {'pickup_latitude': 40.64499,                  
'pickup_longitude': -73.78115,
'trip_distance': 18.38}

nearest_three_neighbors = nearest_neighbors(new_trip, cleaned_trips or [], number = 3)
nearest_three_neighbors
# [{'distance_from_selected': 0.0004569288784918792,
#   'pickup_latitude': 40.64483,
#   'pickup_longitude': -73.781578,
#   'trip_distance': 7.78},
#  {'distance_from_selected': 0.0011292165425673159,
#   'pickup_latitude': 40.644657,
#   'pickup_longitude': -73.782229,
#   'trip_distance': 12.7},
#  {'distance_from_selected': 0.0042359798158141185,
#   'pickup_latitude': 40.648509,
#   'pickup_longitude': -73.783508,
#   'trip_distance': 17.3}]

[{'trip_distance': 1.3,
  'pickup_latitude': 40.766931,
  'pickup_longitude': -73.982098,
  'distance_from_selected': 0.23505256047318146},
 {'trip_distance': 4.5,
  'pickup_latitude': 40.77773,
  'pickup_longitude': -73.951902,
  'distance_from_selected': 0.23505256047318146},
 {'trip_distance': 2.4,
  'pickup_latitude': 40.795678,
  'pickup_longitude': -73.971049,
  'distance_from_selected': 0.23505256047318146}]